# Notebook 2: MED-PC Calculating Latencies

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

In [5]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (10,6)

# Loading in Recording and Metadata from Previous Notebook

# NOTE: If you changed the directory of where the MED-PC recording dataframes, then you must change the directories in the following paths

- This will get all the files in each specified path. The `*` is called a wildcard, and it can be replaced with any number of characters.
    - For more information: https://www.malikbrowne.com/blog/a-beginners-guide-glob-patterns

In [6]:
concatted_medpc_files = glob.glob("./proc/extracted_recording_data_and_metadata/*/MEDPC_recording_*.csv")
metadata_files = glob.glob("./proc/extracted_recording_data_and_metadata/*/metadata_*.csv")
recording_and_metadata_files = glob.glob("./proc/extracted_recording_data_and_metadata/*/recording_metadata_*.csv")

# NOTE: If there is more than one MED-PC recording dataframes, then you must manually change the path in the `pd.read_csv()` for the corresponding variables. Verify if this is the correct file that you want to use

In [7]:
metadata_files[0]

'./proc/extracted_recording_data_and_metadata/experiment_C57_VS_CD1_COMPARISON_cage_1_2_3_4_5_6_date_20220718_20220719/metadata_cage_1_2_3_4_5_6_date_20220718_20220719.csv'

In [8]:
concatted_medpc_files[0]

'./proc/extracted_recording_data_and_metadata/experiment_C57_VS_CD1_COMPARISON_cage_1_2_3_4_5_6_date_20220718_20220719/MEDPC_recording_cage_1_2_3_4_5_6_date_20220718_20220719.csv'

In [9]:
recording_and_metadata_files[0]

'./proc/extracted_recording_data_and_metadata/experiment_C57_VS_CD1_COMPARISON_cage_1_2_3_4_5_6_date_20220718_20220719/recording_metadata_cage_1_2_3_4_5_6_date_20220718_20220719.csv'

In [10]:
metadata_df = pd.read_csv(metadata_files[0], index_col=0)
concatted_medpc_df = pd.read_csv(concatted_medpc_files[0], index_col=0)
recording_and_metadata_df = pd.read_csv(recording_and_metadata_files[0], index_col=0)

- The Dataframe the contains the metadata(columns) of each recording file(rows)
    - We will mostly use this to get the cage number for each subject

In [11]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int
0,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,07/19/22,07/19/22,1,C57_vs_CD1_Comparison,Cage 5,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719
1,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 4.txt,07/18/22,07/18/22,4,C57_vs_CD1_Comparison,Cage 6,4,15:10:30,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718
2,./data/textfiles/2022-07-18_11h27m_Subject 3.txt,C:\MED-PC\Data\2022-07-18_11h27m_Subject 3.txt,07/18/22,07/18/22,3,C57_vs_CD1_Comparison,Cage3,3,11:27:37,12:35:25,C57_reward_training,3,2022-07-18,20220718,2022-07-18,20220718
3,./data/textfiles/2022-07-18_14h04m_Subject 1.txt,C:\MED-PC\Data\2022-07-18_14h04m_Subject 1.txt,07/18/22,07/18/22,1,C57_vs_CD1_comparison,Cage 5,1,14:04:40,15:08:03,CD1_reward_training,5,2022-07-18,20220718,2022-07-18,20220718
4,./data/textfiles/2022-07-18_11h27m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_11h27m_Subject 4.txt,07/18/22,07/18/22,4,C57_vs_CD1_Comparison,Cage3,4,11:27:37,12:35:25,C57_reward_training,3,2022-07-18,20220718,2022-07-18,20220718


- The Dataframe the contains the recording data for all the files.
    - The 1st row is the first data entries for each type of data(with the type being specified by the column label). This does not mean that all these are related or occured for the same trial. i.e. There can be many port entries and exits for a subject before and after a tone is played.
    - Each recording session will usually have a few thousand rows(although, most of of the columns will probably be blank by the end). And then after the last row of one session, the next row will be the next session with a different `file_path`

In [12]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,70.30,64.0,399.0,0.0,60.01,70.67,1.0,49.0,41.0,13.0,59.0,41.0,13.0,20220719,1,./data/textfiles/2022-07-19_13h38m_Subject 1.txt
1,70.72,144.0,399.0,0.0,140.01,71.28,1.0,9.0,43.0,13.0,59.0,41.0,13.0,20220719,1,./data/textfiles/2022-07-19_13h38m_Subject 1.txt
2,71.33,234.0,399.0,0.0,230.01,72.38,1.0,39.0,44.0,13.0,0.0,42.0,13.0,20220719,1,./data/textfiles/2022-07-19_13h38m_Subject 1.txt
3,72.62,314.0,399.0,0.0,310.01,72.70,1.0,59.0,45.0,13.0,1.0,42.0,13.0,20220719,1,./data/textfiles/2022-07-19_13h38m_Subject 1.txt
4,72.73,389.0,399.0,0.0,385.01,73.77,1.0,14.0,47.0,13.0,1.0,42.0,13.0,20220719,1,./data/textfiles/2022-07-19_13h38m_Subject 1.txt


- This dataframe combines the recording data and metadata dataframes. This is done by using the subject ID as a common column to merge together off of.

In [13]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Group,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int
0,70.30,64.0,399.0,0.0,60.01,70.67,1.0,49.0,41.0,13.0,...,Cage 5,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719
1,70.72,144.0,399.0,0.0,140.01,71.28,1.0,9.0,43.0,13.0,...,Cage 5,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719
2,71.33,234.0,399.0,0.0,230.01,72.38,1.0,39.0,44.0,13.0,...,Cage 5,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719
3,72.62,314.0,399.0,0.0,310.01,72.70,1.0,59.0,45.0,13.0,...,Cage 5,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719
4,72.73,389.0,399.0,0.0,385.01,73.77,1.0,14.0,47.0,13.0,...,Cage 5,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719


# Creating a new column that combines cage number and subject

- This is needed because previously the code would treat subjects that have the same ID name as the same, even if the cages were different

In [14]:
recording_and_metadata_df["cage_and_subject"] = recording_and_metadata_df.apply(lambda x: "cage_{}_subject_{}".format(x["cage"], x["Subject"]), axis=1)

In [15]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
0,70.30,64.0,399.0,0.0,60.01,70.67,1.0,49.0,41.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
1,70.72,144.0,399.0,0.0,140.01,71.28,1.0,9.0,43.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2,71.33,234.0,399.0,0.0,230.01,72.38,1.0,39.0,44.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
3,72.62,314.0,399.0,0.0,310.01,72.70,1.0,59.0,45.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
4,72.73,389.0,399.0,0.0,385.01,73.77,1.0,14.0,47.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1


# Getting the Cage Numbers and the Dates for the file names

- Seeing if any rows have NaN values for the cages

In [16]:
recording_and_metadata_df[pd.isna(recording_and_metadata_df["cage"])]

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject


- Dropping the NaN values so we can get the cage numbers

In [17]:
recording_and_metadata_df = recording_and_metadata_df.dropna(subset=["cage"])

# Adjusting the days and cages that are being looked at

- List of all the subject names

In [18]:
all_subjects = sorted(recording_and_metadata_df["subject"].unique())

In [19]:
all_subjects

[1, 2, 3, 4]

- List of all the cage numbers

In [20]:
all_cages = sorted(recording_and_metadata_df["cage"].unique())

In [21]:
all_cages

[1, 2, 3, 4, 5, 6]

- List of all the dates

In [29]:
all_dates = sorted(recording_and_metadata_df["end_date_int"].unique())

In [30]:
all_dates

[20220718, 20220719]

# NOTE: Follow directions below if you want to change which cages or dates to plot

- You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
- Then from above, copy the numbers that you want to use for the cage numbers and the dates

In [31]:
recording_and_metadata_df = recording_and_metadata_df[recording_and_metadata_df["cage"].isin(all_cages) & recording_and_metadata_df["date"].isin(all_dates)]

In [32]:
recording_and_metadata_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
0,70.30,64.0,399.0,0.0,60.01,70.67,1.0,49.0,41.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
1,70.72,144.0,399.0,0.0,140.01,71.28,1.0,9.0,43.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2,71.33,234.0,399.0,0.0,230.01,72.38,1.0,39.0,44.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
3,72.62,314.0,399.0,0.0,310.01,72.70,1.0,59.0,45.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
4,72.73,389.0,399.0,0.0,385.01,73.77,1.0,14.0,47.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,11:27:37,12:35:25,C57_reward_training,3,2022-07-18,20220718,2022-07-18,20220718,cage_3_subject_2
110524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,11:27:37,12:35:25,C57_reward_training,3,2022-07-18,20220718,2022-07-18,20220718,cage_3_subject_2
110525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,11:27:37,12:35:25,C57_reward_training,3,2022-07-18,20220718,2022-07-18,20220718,cage_3_subject_2
110526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,11:27:37,12:35:25,C57_reward_training,3,2022-07-18,20220718,2022-07-18,20220718,cage_3_subject_2


# Getting the Cage Numbers and the Dates to name the files

In [34]:
# removing blank spaces
cage_numbers = [str(int(number)) for number in all_cages if number is not np.nan]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [35]:
cage_numbers_for_title

'1_2_3_4_5_6'

In [36]:
# Getting the first and last recording date to get a range
earliest_date = min(all_dates)
latest_date = max(all_dates)

In [37]:
earliest_date

20220718

In [38]:
latest_date

20220719

# Getting the times that the tones are being played

- Getting all the rows that correspond to one of the files paths. This will be the equivalent of getting all the rows for one session

In [39]:
example_file = recording_and_metadata_df["file_path"].unique()[0]

In [40]:
example_file

'./data/textfiles/2022-07-19_13h38m_Subject 1.txt'

In [41]:
example_one_session_recording_df = recording_and_metadata_df[recording_and_metadata_df["file_path"] == example_file]

- Everything looks normal enough

In [42]:
example_one_session_recording_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
0,70.30,64.0,399.0,0.0,60.01,70.67,1.0,49.0,41.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
1,70.72,144.0,399.0,0.0,140.01,71.28,1.0,9.0,43.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2,71.33,234.0,399.0,0.0,230.01,72.38,1.0,39.0,44.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
3,72.62,314.0,399.0,0.0,310.01,72.70,1.0,59.0,45.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
4,72.73,389.0,399.0,0.0,385.01,73.77,1.0,14.0,47.0,13.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1


- `(S)CSpresentation` is the time that the tone starts playing. MED-PC starts inputting in random numbers that are in the 1000's. So we will remove these

In [43]:
example_one_session_recording_df[50:60]

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
50,400.00,NaN,NaN,0.0,12000.0,400.03,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
51,400.08,NaN,NaN,0.0,9000.0,400.64,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
52,400.68,NaN,NaN,0.0,8500.0,400.71,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
53,402.81,NaN,NaN,0.0,9000.0,403.37,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
54,403.39,NaN,NaN,0.0,9500.0,407.12,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
55,409.00,NaN,NaN,0.0,12000.0,409.77,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
56,409.79,NaN,NaN,0.0,8000.0,409.87,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
57,409.93,NaN,NaN,0.0,9500.0,409.99,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
58,410.72,NaN,NaN,0.0,8000.0,410.86,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
59,411.04,NaN,NaN,0.0,9000.0,411.23,0.0,0.0,0.0,0.0,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1


- `(S)CSpresentation` then starts adding blanks(NaN) so we will remove these too

In [44]:
example_one_session_recording_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
2507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1


- We will use a function that removes the `NaN` and the tone times that are divisible by 1000. This will get the tones time that were actually used for the recordings

In [45]:
example_one_session_valid_tones = processing.tone.get_valid_tones(example_one_session_recording_df["(S)CSpresentation"])

In [46]:
example_one_session_valid_tones

0       60.01
1      140.01
2      230.01
3      310.01
4      385.01
5      485.01
6      580.01
7      670.01
8      750.01
9      840.01
10     940.01
11    1030.01
12    1150.01
13    1240.01
14    1325.01
15    1415.01
16    1510.01
17    1630.01
18    1710.01
19    1805.01
20    1885.01
21    1975.01
22    2055.01
23    2130.01
24    2230.01
25    2325.01
26    2415.01
27    2495.01
28    2585.01
29    2685.01
30    2775.01
31    2895.01
32    2985.01
33    3070.01
34    3160.01
35    3255.01
36    3345.01
37    3425.01
38    3515.01
Name: (S)CSpresentation, dtype: float64

# Getting the First Port Entry After Each Tone

- With these tone playing times, we will get the first port entry that comes after the tone playing time
    - This was done by getting all the port entries that came after the tone. And then getting the port entry time that was earliest in time(aka smallest number)

In [47]:
processing.tone.get_first_port_entries_after_tone(tone_pd_series=example_one_session_valid_tones, port_entries_pd_series=example_one_session_recording_df["(P)Portentry"], port_exits_pd_series=example_one_session_recording_df["(N)Portexit"])

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone
0,60.01,70.30,70.67
1,140.01,150.59,151.52
2,230.01,242.38,245.32
3,310.01,312.96,313.35
4,385.01,400.00,400.03
5,485.01,487.24,487.29
6,580.01,587.95,594.75
7,670.01,688.32,688.52
8,750.01,754.79,761.70
9,840.01,844.59,851.95


- Getting the first port entry times for all the sessions

In [48]:
concatted_first_porty_entry_dataframe = processing.tone.get_concatted_first_porty_entry_after_tone_dataframe(concatted_medpc_df=recording_and_metadata_df)

In [49]:
concatted_first_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1


In [50]:
concatted_first_porty_entry_dataframe.tail()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject
1711,3160.01,3516.34,3523.1,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1712,3255.01,3516.34,3523.1,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1713,3345.01,3516.34,3523.1,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1714,3425.01,3516.34,3523.1,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1715,3515.01,3516.34,3523.1,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2


# Getting the Last Port Entry Before the Tone

- We will do the same, but get the port entries that happened before the tone

In [51]:
port_entries_pd_series = example_one_session_recording_df["(P)Portentry"]
port_exit_pd_series = example_one_session_recording_df["(N)Portexit"]

- Example run for one session

In [52]:
example_last_port_entries_before_tone = processing.tone.get_last_port_entries_before_tone(tone_pd_series=example_one_session_valid_tones, port_entries_pd_series=port_entries_pd_series, port_exits_pd_series=port_exit_pd_series)

In [53]:
example_last_port_entries_before_tone

,current_tone_time,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone
0,60.01,NaN,NaN
1,140.01,103.53,103.57
2,230.01,190.14,190.86
3,310.01,281.07,284.20
4,385.01,360.55,362.00
5,485.01,439.24,440.26
6,580.01,542.56,544.89
7,670.01,626.85,628.97
8,750.01,733.38,733.50
9,840.01,792.06,793.76


- Getting all the port entries that happen before the tone for all sessions

In [54]:
concatted_last_porty_entry_dataframe = processing.tone.get_concatted_last_porty_entry_before_tone_dataframe(concatted_medpc_df=recording_and_metadata_df)

In [55]:
concatted_last_porty_entry_dataframe

,current_tone_time,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,file_path,date,subject
0,60.01,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
1,140.01,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
2,230.01,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
3,310.01,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
4,385.01,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1
...,...,...,...,...,...,...
1711,3160.01,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1712,3255.01,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1713,3345.01,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2
1714,3425.01,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2


## Merging the dataframes for the first port entry after the tone and the last tone before the tone 

In [56]:
# Combining it based on the tone times and the file path
concatted_first_and_last_porty_entry_dataframe = pd.merge(concatted_first_porty_entry_dataframe, concatted_last_porty_entry_dataframe,  how='left', left_on=['current_tone_time','file_path'], right_on = ['current_tone_time','file_path'], suffixes=('', '_y'))
# Dropping all columns that are duplicates
concatted_first_and_last_porty_entry_dataframe = concatted_first_and_last_porty_entry_dataframe.drop(concatted_first_and_last_porty_entry_dataframe.filter(regex='_y$').columns.tolist(), axis=1)

In [57]:
# Adding the metadata
concatted_first_and_last_porty_entry_dataframe = pd.merge(concatted_first_and_last_porty_entry_dataframe, metadata_df,  how='left', left_on=['file_path'], right_on = ["index"], suffixes=('', '_y'))
# Making a new column for cage and subject combined
concatted_first_and_last_porty_entry_dataframe["cage_and_subject"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda x: "cage_{}_subject_{}".format(x["cage"], x["Subject"]), axis=1)

In [58]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,2,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1


In [59]:
concatted_first_and_last_porty_entry_dataframe.shape

(1716, 25)

- Getting all the rows that have port entry to port exit time that overlaps with the tone playing
    - This would be the port entries that came before the tone, but had a port exit after

In [60]:
port_entries_that_overlap = concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["port_exit_after_last_port_entry_before_tone"] >= concatted_first_and_last_porty_entry_dataframe["current_tone_time"]]

In [61]:
port_entries_that_overlap.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,Box,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject
199,385.01,424.71,427.85,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,379.19,385.50,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,2,12:27:39,13:29:52,C57_reward_training,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1
202,670.01,727.85,729.52,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,667.64,672.62,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,2,12:27:39,13:29:52,C57_reward_training,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1
221,2415.01,2415.41,2415.44,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,2414.58,2415.28,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,2,12:27:39,13:29:52,C57_reward_training,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1
232,3425.01,3425.01,3425.90,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,3425.01,3425.90,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,2,12:27:39,13:29:52,C57_reward_training,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1
510,310.01,361.24,361.30,./data/textfiles/2022-07-18_15h10m_Subject 3.txt,20220718,3,309.95,310.46,./data/textfiles/2022-07-18_15h10m_Subject 3.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 3.txt,...,3,15:10:30,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_3


In [62]:
port_entries_that_overlap.shape

(33, 25)

# Getting the Latency for Each Session of a Tone Playing

- To calculate latency, we would just subtract the time of the first port entry to that of the tone playing

In [63]:
concatted_first_and_last_porty_entry_dataframe["latency"] = concatted_first_porty_entry_dataframe["first_port_entry_after_tone"] - concatted_first_and_last_porty_entry_dataframe["current_tone_time"]

In [64]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,Start Time,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,10.29
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,10.58
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,12.37
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,2.95
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,13:38:55,14:41:48,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,14.99


- For all the latencies that are are greater than 30 seconds, we changed the latency to 30 seconds

In [65]:
concatted_first_and_last_porty_entry_dataframe["latency_adjusted_greater_than_30"] = concatted_first_and_last_porty_entry_dataframe["latency"].apply(lambda x: 30 if x >= 30 else x)

In [66]:
concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["latency"] >= 30].head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,End Time,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency,latency_adjusted_greater_than_30
39,60.01,148.32,148.55,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,20220718,4,NaN,NaN,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 4.txt,...,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_4,88.31,30.0
41,230.01,302.88,302.99,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,20220718,4,224.21,224.28,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 4.txt,...,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_4,72.87,30.0
42,310.01,370.12,370.23,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,20220718,4,303.05,303.09,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 4.txt,...,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_4,60.11,30.0
43,385.01,553.87,553.93,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,20220718,4,370.12,370.23,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 4.txt,...,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_4,168.86,30.0
44,485.01,553.87,553.93,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,20220718,4,370.12,370.23,./data/textfiles/2022-07-18_15h10m_Subject 4.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 4.txt,...,16:20:28,CD1_reward_training,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_4,68.86,30.0


# Seeing which port entries and port exit durations overlap with the tone

- Making a column that states whether or not the last port entry before the tone has a port exit after the tone. This would mean that they are overlapping.

In [67]:
concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: True if row["port_exit_after_last_port_entry_before_tone"] >= row["current_tone_time"] else False, axis=1)


In [68]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,MSN,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,10.29,10.29,False
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,10.58,10.58,False
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,12.37,12.37,False
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,2.95,2.95,False
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,CD1_reward_training,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,14.99,14.99,False


- Comparing the latencies of the first port entry after the tone between these two groups:
    - 1. Those with the last port entry before the tone that has a port exit after the tone("overlapping" group)
    - 2. Those with the last port entry before the tone that has a port exit before the tone("nonoverlapping" group)

In [69]:
overlapping_df = concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]]

In [70]:
non_overlapping_df = concatted_first_and_last_porty_entry_dataframe[~concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]]

- Number of rows of overlapping and nonoverlapping last port entries before the tone

In [71]:
overlapping_df.shape

(33, 28)

In [72]:
non_overlapping_df.shape

(1683, 28)

- Latency between the tone playing and the first port entry after the tone

In [73]:
overlapping_df["latency"].mean()

11.881818181818073

In [74]:
non_overlapping_df["latency"].mean()

79.75494203782766

- Latency between the tone playing and the first port entry after the tone. This had been adjusted so that all latencies that are greater than 30 seconds were adjusted to be just 30 seconds.

In [75]:
overlapping_df["latency_adjusted_greater_than_30"].mean()

9.59242424242413

In [76]:
non_overlapping_df["latency_adjusted_greater_than_30"].mean()

21.557382550335507

# Adjusting the Latencies if the Port Entries/Exit Overlap With the Tone Times

- For all the rows with overlapping last port entries before the tone, we will adjust the latency of the tone to the first port entry after the tone to 0. (Because the first port entry after the tone would be considered to be the same as the last port entry before the tone)

In [77]:
concatted_first_and_last_porty_entry_dataframe["latency_adjusted_overlap"] = concatted_first_and_last_porty_entry_dataframe.apply(lambda row: 0 if row["overlapping_port_entry_with_tone"] else row["latency_adjusted_greater_than_30"], axis=1)

In [78]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,10.29,10.29,False,10.29
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,10.58,10.58,False,10.58
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,12.37,12.37,False,12.37
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,2.95,2.95,False,2.95
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,5,2022-07-19,20220719,2022-07-19,20220719,cage_5_subject_1,14.99,14.99,False,14.99


- All the rows with overlapping last port entries before the tone

In [79]:
concatted_first_and_last_porty_entry_dataframe[concatted_first_and_last_porty_entry_dataframe["overlapping_port_entry_with_tone"]].head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,cage,start_date_datetime,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap
199,385.01,424.71,427.85,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,379.19,385.50,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1,39.70,30.0,True,0.0
202,670.01,727.85,729.52,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,667.64,672.62,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1,57.84,30.0,True,0.0
221,2415.01,2415.41,2415.44,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,2414.58,2415.28,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1,0.40,0.4,True,0.0
232,3425.01,3425.01,3425.90,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,20220719,1,3425.01,3425.90,./data/textfiles/2022-07-19_12h27m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_12h27m_Subject 1.txt,...,4,2022-07-19,20220719,2022-07-19,20220719,cage_4_subject_1,0.00,0.0,True,0.0
510,310.01,361.24,361.30,./data/textfiles/2022-07-18_15h10m_Subject 3.txt,20220718,3,309.95,310.46,./data/textfiles/2022-07-18_15h10m_Subject 3.txt,C:\MED-PC\Data\2022-07-18_15h10m_Subject 3.txt,...,6,2022-07-18,20220718,2022-07-18,20220718,cage_6_subject_3,51.23,30.0,True,0.0


## Labeling if a row's latency is less than 10 seconds or not

- We will be getting the ratio of latencies(from the time that the tone played, to the first port entry after the tone) that are less than 10 seconds for each session. So we will label all the adjusted latencies that are less than 10 seconds as `True`, and the those with latencies that are greater than 10 seconds as `False` 

In [80]:
concatted_first_and_last_porty_entry_dataframe["adjusted_30_second_latency_less_than_10_seconds_ratio"] = concatted_first_and_last_porty_entry_dataframe["latency_adjusted_greater_than_30"].apply(lambda x: True if x <= 10 else False)

In [81]:
concatted_first_and_last_porty_entry_dataframe["overlap_adjusted_latency_less_than_10_seconds"] = concatted_first_and_last_porty_entry_dataframe["latency_adjusted_overlap"].apply(lambda x: True if x <= 10 else False)

In [82]:
concatted_first_and_last_porty_entry_dataframe.head()

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap,adjusted_30_second_latency_less_than_10_seconds_ratio,overlap_adjusted_latency_less_than_10_seconds
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,10.29,10.29,False,10.29,False,False
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,10.58,10.58,False,10.58,False,False
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,12.37,12.37,False,12.37,False,False
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,2.95,2.95,False,2.95,True,True
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,14.99,14.99,False,14.99,False,False


# Making Dataframes that have Each Subject as the Row and Each Date as the Column

- Pivot plots allow us to make columns out of each unique entry in a selected column. (For this, it will be the date column that we will turn into columns) The index for the rows will be each unique entry in another column. (For this, it will be the subject's ID) And the values will be the aggregate of the values that have both the values in the previous two selected columns. (For this it will be the latency columns and we will take the mean)
    - https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

- Making a folder for all the pivotplots

# NOTE: If you decide to change the cage numbers or the dates used from the original ones from the imported dataframes, you will need to change the subsequent path to the cage number and dates you are using

In [83]:
first_port_entry_to_tone_latency_output_directory = "./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [84]:
first_port_entry_to_tone_latency_output_directory

'./proc/subject_rows_with_date_columns_dataframes/first_port_entry_to_tone_latency/cage_1_2_3_4_5_6_date_20220718_20220719'

In [85]:
os.makedirs(first_port_entry_to_tone_latency_output_directory, exist_ok=True)

In [86]:
latencies_less_than_10_seconds_ratio_output_directory = "./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [87]:
latencies_less_than_10_seconds_ratio_output_directory

'./proc/subject_rows_with_date_columns_dataframes/latencies_less_than_10_seconds_ratio/cage_1_2_3_4_5_6_date_20220718_20220719'

In [88]:
os.makedirs(latencies_less_than_10_seconds_ratio_output_directory, exist_ok=True)

## Making a pivot plot with the original latency

In [90]:
concatted_first_and_last_porty_entry_dataframe

,current_tone_time,first_port_entry_after_tone,port_exit_after_first_port_entry_after_tone,file_path,date,subject,last_port_entry_before_tone,port_exit_after_last_port_entry_before_tone,index,File,...,start_date_int,end_date_datetime,end_date_int,cage_and_subject,latency,latency_adjusted_greater_than_30,overlapping_port_entry_with_tone,latency_adjusted_overlap,adjusted_30_second_latency_less_than_10_seconds_ratio,overlap_adjusted_latency_less_than_10_seconds
0,60.01,70.30,70.67,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,NaN,NaN,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,10.29,10.29,False,10.29,False,False
1,140.01,150.59,151.52,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,103.53,103.57,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,10.58,10.58,False,10.58,False,False
2,230.01,242.38,245.32,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,190.14,190.86,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,12.37,12.37,False,12.37,False,False
3,310.01,312.96,313.35,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,281.07,284.20,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,2.95,2.95,False,2.95,True,True
4,385.01,400.00,400.03,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,20220719,1,360.55,362.00,./data/textfiles/2022-07-19_13h38m_Subject 1.txt,C:\MED-PC\Data\2022-07-19_13h38m_Subject 1.txt,...,20220719,2022-07-19,20220719,cage_5_subject_1,14.99,14.99,False,14.99,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,3160.01,3516.34,3523.10,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,C:\MED-PC\Data\2022-07-18_11h27m_Subject 2.txt,...,20220718,2022-07-18,20220718,cage_3_subject_2,356.33,30.00,False,30.00,False,False
1712,3255.01,3516.34,3523.10,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,C:\MED-PC\Data\2022-07-18_11h27m_Subject 2.txt,...,20220718,2022-07-18,20220718,cage_3_subject_2,261.33,30.00,False,30.00,False,False
1713,3345.01,3516.34,3523.10,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,C:\MED-PC\Data\2022-07-18_11h27m_Subject 2.txt,...,20220718,2022-07-18,20220718,cage_3_subject_2,171.33,30.00,False,30.00,False,False
1714,3425.01,3516.34,3523.10,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,20220718,2,2723.54,2723.59,./data/textfiles/2022-07-18_11h27m_Subject 2.txt,C:\MED-PC\Data\2022-07-18_11h27m_Subject 2.txt,...,20220718,2022-07-18,20220718,cage_3_subject_2,91.33,30.00,False,30.00,False,False


In [91]:
all_latency_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency', \
                                        index=['cage_and_subject'], columns=['end_date_int'], aggfunc=np.mean)


In [94]:
all_latency_pivot_plot.head()

end_date_int,20220718,20220719
cage_and_subject,,
cage_1_subject_1,157.351538,337.954583
cage_1_subject_2,175.416667,171.389487
cage_1_subject_3,56.433333,65.062667
cage_1_subject_4,NaN,53.292632
cage_2_subject_1,169.048649,50.398974


In [95]:
all_latency_pivot_plot.tail()

end_date_int,20220718,20220719
cage_and_subject,,
cage_5_subject_4,43.978718,47.938718
cage_6_subject_1,90.394103,NaN
cage_6_subject_2,108.742564,NaN
cage_6_subject_3,13.220256,NaN
cage_6_subject_4,243.953333,28.704872


- Getting the cage numbers for each subject and then making a dictionary out of it
    - This will be used to add the cage information to the pivot plots

- Adding the cage information with the dictionary

In [ ]:
file_path = "original_latency_cage_{}_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    all_latency_pivot_plot.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    all_latency_pivot_plot.to_csv(output_path)

## Making a pivot plot with the latency that were greater than 30 seconds being adjusted 

In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_greater_than_30', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot["cage"] = latency_greater_than_30_seconds_adjusted_pivot_plot.index.map(subject_to_cage)

In [ ]:
latency_greater_than_30_seconds_adjusted_pivot_plot.head()

In [ ]:
file_path = "subject_to_date_latency_greater_than_30_seconds_adjusted_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    latency_greater_than_30_seconds_adjusted_pivot_plot.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    latency_greater_than_30_seconds_adjusted_pivot_plot.to_csv(output_path)

## Making a pivot plot with adjusted latencies that overlapped with a tone playing

In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='latency_adjusted_overlap', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot["cage"] = latency_that_overlaps_with_tone_adjusted_pivot_plot.index.map(subject_to_cage)

In [ ]:
latency_that_overlaps_with_tone_adjusted_pivot_plot.head()

In [ ]:
file_path = "subject_to_date_latency_that_overlaps_with_tone_adjusted_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    latency_that_overlaps_with_tone_adjusted_pivot_plot.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    latency_that_overlaps_with_tone_adjusted_pivot_plot.to_csv(output_path)

## Making a pivot plot with the ratio of original latencies that are less than 10 seconds

In [ ]:
original_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='adjusted_30_second_latency_less_than_10_seconds_ratio', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
original_less_than_10_latency_df["cage"] = original_less_than_10_latency_df.index.map(subject_to_cage)

In [ ]:
original_less_than_10_latency_df

In [ ]:
file_path = "subject_to_date_less_than_10_seconds_original_latencies_ratios_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)
output_path = os.path.join(first_port_entry_to_tone_latency_output_directory, file_path)
try:
    original_less_than_10_latency_df.to_csv(output_path)
except:
    output_path = os.path.abspath(output_path)
    # Changing path if using on Windows because it raises an error if the path is longer than 260 characters
    # But adding this suffix bypasses this
    output_path = u"\\\\?\\" + output_path
    original_less_than_10_latency_df.to_csv(output_path)

## Making a pivot plot with the ratio of overlap adjusted latencies that are less than 10 seconds

In [ ]:
overlap_adjusted_less_than_10_latency_df = pd.pivot_table(concatted_first_and_last_porty_entry_dataframe, values='overlap_adjusted_latency_less_than_10_seconds', index=['subject'],

                    columns=['date'], aggfunc=np.mean)


In [ ]:
overlap_adjusted_less_than_10_latency_df["cage"] = overlap_adjusted_less_than_10_latency_df.index.map(subject_to_cage)

In [ ]:
overlap_adjusted_less_than_10_latency_df

In [ ]:
overlap_adjusted_less_than_10_latency_df.to_csv(os.path.join(latencies_less_than_10_seconds_ratio_output_directory, "subject_to_date_less_than_10_seconds_overlap_adjusted_latencies_ratios_cage_{}_date_{}_{}.csv".format(cage_numbers_for_title, earliest_date, latest_date)))
overlap_adjusted_less_than_10_latency_df.to_excel(os.path.join(latencies_less_than_10_seconds_ratio_output_directory, "subject_to_date_less_than_10_seconds_overlap_adjusted_latencies_ratios_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

# Plotting the latencies

## Getting the averages for all the trials in one subject and one day

- We will plot a line for each subject. With the X-axis being the days since the first session. And the Y-axis is the latency values averaged across all the trials for one recording session
- Grouping all the rows with the same subject and date(aka, all the trials in one session). And then getting the mean for each value

In [ ]:
grouped_averaged_first_porty_entry_dataframe = concatted_first_and_last_porty_entry_dataframe.groupby(by=["file_path", "date", "subject"]).mean()

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

## Reformatting the Dataframe for plotting

- Resetting the index so that we can use the data as a value

In [ ]:
grouped_averaged_first_porty_entry_dataframe = grouped_averaged_first_porty_entry_dataframe.reset_index()

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

- Turning the date column which is a string, into an integer
    - This will allow us to use the date as the X-axis for plotting

In [ ]:
grouped_averaged_first_porty_entry_dataframe["date_int"] = grouped_averaged_first_porty_entry_dataframe["date"].astype(int)

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

- Adding the cage information
    - We will make one plot for each cage

In [ ]:
grouped_averaged_first_porty_entry_dataframe["cage"] = grouped_averaged_first_porty_entry_dataframe["subject"].map(subject_to_cage).astype(int)

In [ ]:
grouped_averaged_first_porty_entry_dataframe.head()

## Plotting the latencies that were adjusted if they greater than 30 seconds

In [ ]:
original_average_latency_output_directory = "./proc/plots/original_average_latency_plots/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
original_average_latency_output_directory

In [ ]:
os.makedirs(original_average_latency_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["latency_adjusted_greater_than_30"], '-o', label=subject)

    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 30)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Adjusted Average Latency of First Entry to Tone Onset")
    ax.set_title("Latency of Port Entry to Tone: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    file_name = "original_average_latency_plot_cage_{}_date_{}_{}.png".format(cage, earliest_date, latest_date)
    plt.savefig(os.path.join(original_average_latency_output_directory, file_name))

## Plotting the original ratio of latencies that are less than 10 seconds

In [ ]:
less_than_10_second_original_latency_ratio_plots_output_directory = "./proc/plots/less_than_10_second_original_latency_ratio/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
less_than_10_second_original_latency_ratio_plots_output_directory

In [ ]:
os.makedirs(less_than_10_second_original_latency_ratio_plots_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["adjusted_30_second_latency_less_than_10_seconds_ratio"], '-o', label=subject)
    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 1)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Proportion of Latencies")
    ax.set_title("Less Than 10 Seconds Latencies from Tone Onset: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    
    file_name = "less_than_10_second_original_latency_ratio_plot_cage_{}_date_{}_{}.png".format(cage, earliest_date, latest_date)
    plt.savefig(os.path.join(less_than_10_second_original_latency_ratio_plots_output_directory, file_name))


# Plotting the Adjusted Latencies for Entries that Overlap with the Tone

## Plotting the Latencies 

In [ ]:
overlap_adjusted_average_latency_plots_output_directory = "./proc/plots/overlap_adjusted_average_latency_plots/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
overlap_adjusted_average_latency_plots_output_directory

In [ ]:
os.makedirs(overlap_adjusted_average_latency_plots_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["latency_adjusted_overlap"], '-o', label=subject)

    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 30)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Proportion of Latencies")
    ax.set_title("Overlap Adjusted Latency of Port Entry to Tone: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    file_name = "overlap_adjusted_average_latency_plot_cage_{}_date_{}_{}.png".format(cage, earliest_date, latest_date)

    plt.savefig(os.path.join(overlap_adjusted_average_latency_plots_output_directory, file_name))


# Plotting the ratio of overlap adjusted latencies that are less than 10 seconds

In [ ]:
less_than_10_second_overlap_adjusted_latency_ratio_output_directory = "./proc/plots/less_than_10_second_overlap_adjusted_latency_ratio/cage_{}_date_{}_{}".format(cage_numbers_for_title, earliest_date, latest_date)

In [ ]:
less_than_10_second_overlap_adjusted_latency_ratio_output_directory

In [ ]:
os.makedirs(less_than_10_second_overlap_adjusted_latency_ratio_output_directory, exist_ok=True)

In [ ]:
# Plotting for each cage
for cage in grouped_averaged_first_porty_entry_dataframe["cage"].unique():
    fig, ax = plt.subplots()
    # Getting all the rows with the current cage number
    cage_df = grouped_averaged_first_porty_entry_dataframe[grouped_averaged_first_porty_entry_dataframe["cage"] == cage]
    
    # Plotting a line for each subject
    for subject in cage_df["subject"].unique():
        # Getting all the rows with the current subject
        subject_df = cage_df[cage_df["subject"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        ax.plot(subject_df["date_int"] - subject_df["date_int"].min() + 1, subject_df["overlap_adjusted_latency_less_than_10_seconds"], '-o', label=subject)

    # Setting the Y-Axis to only plot from 0 to 30 because we adjusted the latency to always be under 30
    ax.set_ylim(0, 1)
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("The Days After the First Session")
    ax.set_ylabel("Proportion of Latencies")
    ax.set_title("Overlap Adjusted Less Than 10sec Latencies from Tone: Cage {}".format(cage))
    # To show the legend
    ax.legend()
    
    file_name = "less_than_10_second_overlap_adjusted_latency_ratio_plot_cage{}_date_{}_{}.svg".format(cage, earliest_date, latest_date)
    plt.savefig(os.path.join(less_than_10_second_overlap_adjusted_latency_ratio_output_directory, file_name))
